# Лабораторная работа №2  
## Проведение исследований с логистической и линейной регрессией

В работе исследуются:
- **логистическая регрессия** для задачи классификации;
- **линейная регрессия** для задачи регрессии.


## 0. Импорт библиотек

In [22]:
import os
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from scipy import sparse
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, KFold
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
    root_mean_squared_error, r2_score
)
from sklearn.metrics import make_scorer
rmse_scorer = make_scorer(root_mean_squared_error, greater_is_better=False)

RANDOM_STATE = 42


## 1. Загрузка данных и разбиение выборок

In [23]:
# Пути к данным
LOAN_PATH = os.path.join("data", "Loan_approval_data_2025.csv")
SONG_PATH = os.path.join("data", "song_data.csv")

# Загрузка датасетов
loan_df = pd.read_csv(LOAN_PATH)
song_df = pd.read_csv(SONG_PATH)

# Одобрение кредита
y_cls = loan_df["loan_status"]
X_cls = loan_df.drop(columns=["loan_status", "customer_id"])

cat_cols_cls = X_cls.select_dtypes(include=["object"]).columns.tolist()
num_cols_cls = X_cls.select_dtypes(exclude=["object"]).columns.tolist()

Xc_train, Xc_test, yc_train, yc_test = train_test_split(
    X_cls, y_cls, test_size=0.2, random_state=RANDOM_STATE, stratify=y_cls
)

# Популярность песни
y_reg = song_df["song_popularity"]
X_reg = song_df.drop(columns=["song_popularity", "song_name"])

cat_cols_reg = X_reg.select_dtypes(include=["object"]).columns.tolist()
num_cols_reg = X_reg.select_dtypes(exclude=["object"]).columns.tolist()

Xr_train, Xr_test, yr_train, yr_test = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=RANDOM_STATE
)


## 2. Бейзлайн моделей

Формируем бейзлайн для задач классификации и регрессии с использованием линейных моделей и пайплайнов предобработки данных. Для числовых признаков выполняется заполнение пропусков и масштабирование, а для категориальных — заполнение пропусков и one-hot кодирование. Предобработка объединяется с моделью в единый пайплайн, что обеспечивает корректную обработку данных при обучении и тестировании.

Для классификации используется логистическая регрессия, для регрессии — линейная регрессия. После обучения моделей вычисляются стандартные метрики качества, которые служат отправной точкой для последующего сравнения с улучшенными моделями.



In [24]:
numeric_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocess_cls = ColumnTransformer([
    ("num", numeric_pipe, num_cols_cls),
    ("cat", categorical_pipe, cat_cols_cls)
])

preprocess_reg = ColumnTransformer([
    ("num", numeric_pipe, num_cols_reg),
    ("cat", categorical_pipe, cat_cols_reg)
])

baseline_cls = Pipeline([
    ("prep", preprocess_cls),
    ("model", LogisticRegression(max_iter=2000, solver="liblinear"))
])

baseline_reg = Pipeline([
    ("prep", preprocess_reg),
    ("model", LinearRegression())
])

baseline_cls.fit(Xc_train, yc_train)
baseline_reg.fit(Xr_train, yr_train)

yc_pred = baseline_cls.predict(Xc_test)
yc_proba = baseline_cls.predict_proba(Xc_test)[:, 1]
yr_pred = baseline_reg.predict(Xr_test)

baseline_cls_res = {
    "accuracy": accuracy_score(yc_test, yc_pred),
    "precision": precision_score(yc_test, yc_pred, zero_division=0),
    "recall": recall_score(yc_test, yc_pred, zero_division=0),
    "f1": f1_score(yc_test, yc_pred, zero_division=0),
    "roc_auc": roc_auc_score(yc_test, yc_proba)
}

baseline_reg_res = {
    "rmse": root_mean_squared_error(yr_test, yr_pred),
    "mae": mean_absolute_error(yr_test, yr_pred),
    "r2": r2_score(yr_test, yr_pred)
}

print("Бейзлайн (классификация):", baseline_cls_res)
print("Бейзлайн (регрессия):    ", baseline_reg_res)

Бейзлайн (классификация): {'accuracy': 0.8652, 'precision': 0.8709619846510798, 'recall': 0.8864668483197093, 'f1': 0.8786460208858481, 'roc_auc': np.float64(0.9445712917471123)}
Бейзлайн (регрессия):     {'rmse': 21.48576103236179, 'mae': 17.055369122574042, 'r2': 0.04239734907577464}


## 3. Улучшение бейзлайна

В части классификации подбираем параметр регуляризации `C` для логистической регрессии. Используется `GridSearchCV` с кросс-валидацией `StratifiedKFold`, что позволяет сохранить баланс классов в разбиениях. Качество моделей оценивается по метрике ROC-AUC. После выбора лучшей модели рассчитываются основные метрики классификации на тестовой выборке.

В части регрессии рассматриваются две регуляризованные линейные модели — Ridge и Lasso. Для обеих моделей выполняется подбор параметра регуляризации `alpha` с использованием кросс-валидации `KFold`. В качестве критерия качества используется RMSE. После обучения выбирается модель с наилучшим значением метрики на кросс-валидации и оценивается её качество на тестовой выборке по метрикам RMSE, MAE и R².

Таким образом, данный код реализует этап улучшения моделей за счёт настройки гиперпараметров и позволяет сравнить полученные результаты с бейзлайном.


In [25]:
# --- Логистическая регрессия: подбор C ---
param_grid_cls = {"model__C": [0.01, 0.1, 1, 10]}

tuned_cls = Pipeline([
    ("prep", preprocess_cls),
    ("model", LogisticRegression(max_iter=3000, solver="lbfgs"))
])

cv_cls = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

gs_cls = GridSearchCV(
    tuned_cls,
    param_grid_cls,
    cv=cv_cls,
    scoring="roc_auc",
    n_jobs=1  # важно: на macOS часто убирает матричные RuntimeWarning
)

gs_cls.fit(Xc_train, yc_train)
best_cls = gs_cls.best_estimator_

print("Лучшие параметры (классификация):", gs_cls.best_params_, "| CV ROC-AUC:", gs_cls.best_score_)

yc_pred2 = best_cls.predict(Xc_test)
yc_proba2 = best_cls.predict_proba(Xc_test)[:, 1]

improved_cls_res = {
    "accuracy": accuracy_score(yc_test, yc_pred2),
    "precision": precision_score(yc_test, yc_pred2, zero_division=0),
    "recall": recall_score(yc_test, yc_pred2, zero_division=0),
    "f1": f1_score(yc_test, yc_pred2, zero_division=0),
    "roc_auc": roc_auc_score(yc_test, yc_proba2),
}

print("Улучшенная (классификация):", improved_cls_res)

# --- Регрессия: Ridge и Lasso (подбор alpha) ---
param_grid_reg = {"model__alpha": [0.01, 0.1, 1.0, 10.0]}

ridge_pipe = Pipeline([
    ("prep", preprocess_reg),
    ("model", Ridge())
])

lasso_pipe = Pipeline([
    ("prep", preprocess_reg),
    ("model", Lasso(max_iter=10000))
])

cv_reg = KFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

gs_ridge = GridSearchCV(ridge_pipe, param_grid_reg, cv=cv_reg, scoring=rmse_scorer, n_jobs=1)
gs_lasso = GridSearchCV(lasso_pipe, param_grid_reg, cv=cv_reg, scoring=rmse_scorer, n_jobs=1)

gs_ridge.fit(Xr_train, yr_train)
gs_lasso.fit(Xr_train, yr_train)

use_ridge = gs_ridge.best_score_ > gs_lasso.best_score_
best_reg = gs_ridge.best_estimator_ if use_ridge else gs_lasso.best_estimator_

print("Лучший регрессор:", "Ridge" if use_ridge else "Lasso")
print("Лучшие параметры (регрессия):",
      (gs_ridge.best_params_ if use_ridge else gs_lasso.best_params_),
      "| CV -RMSE:", (gs_ridge.best_score_ if use_ridge else gs_lasso.best_score_))

yr_pred2 = best_reg.predict(Xr_test)

improved_reg_res = {
    "rmse": root_mean_squared_error(yr_test, yr_pred2),
    "mae": mean_absolute_error(yr_test, yr_pred2),
    "r2": r2_score(yr_test, yr_pred2),
}

print("Улучшенная (регрессия):    ", improved_reg_res)


Лучшие параметры (классификация): {'model__C': 10} | CV ROC-AUC: 0.9460661149993392
Улучшенная (классификация): {'accuracy': 0.8653, 'precision': 0.8711174580506962, 'recall': 0.8864668483197093, 'f1': 0.8787251282974701, 'roc_auc': np.float64(0.9445407400896545)}
Лучший регрессор: Lasso
Лучшие параметры (регрессия): {'model__alpha': 0.01} | CV -RMSE: -21.387719777963973
Улучшенная (регрессия):     {'rmse': 21.485203664412715, 'mae': 17.054866331327474, 'r2': 0.04244703129106686}


## 4. Подготовка матриц для своих моделей

In [26]:
# сигмоида
def sigmoid(z):
    z = np.clip(z, -50, 50)
    return 1.0 / (1.0 + np.exp(-z))



# --- Классификация: loan_status ---
Xc_train_tr = preprocess_cls.fit_transform(Xc_train)
Xc_test_tr  = preprocess_cls.transform(Xc_test)

# y в numpy
yc_train_np = np.asarray(yc_train).astype(int)
yc_test_np  = np.asarray(yc_test).astype(int)

print("CLS shapes:", Xc_train_tr.shape, Xc_test_tr.shape, "| sparse:", sparse.issparse(Xc_train_tr))

# --- Регрессия: song_popularity ---
Xr_train_tr = preprocess_reg.fit_transform(Xr_train)
Xr_test_tr  = preprocess_reg.transform(Xr_test)

yr_train_np = np.asarray(yr_train).astype(float)
yr_test_np  = np.asarray(yr_test).astype(float)

# для OLS удобнее плотный формат (здесь обычно немного признаков)
Xr_train_arr = Xr_train_tr.toarray() if sparse.issparse(Xr_train_tr) else np.asarray(Xr_train_tr)
Xr_test_arr  = Xr_test_tr.toarray()  if sparse.issparse(Xr_test_tr)  else np.asarray(Xr_test_tr)

print("REG shapes:", Xr_train_arr.shape, Xr_test_arr.shape)


CLS shapes: (40000, 27) (10000, 27) | sparse: False
REG shapes: (15068, 13) (3767, 13)


## 5. Имплементация линейной регрессии (OLS) + Ridge

In [27]:
class MyLinearRegression:
    def __init__(self):
        self.coef_ = None
        self.intercept_ = 0.0

    def fit(self, X, y):
        X = np.asarray(X, dtype=float)
        y = np.asarray(y, dtype=float)

        # добавляем столбец единиц под свободный член
        Xb = np.c_[np.ones((X.shape[0], 1)), X]
        # OLS через псевдообратную (устойчивее, чем явная (X^T X)^-1)
        w = np.linalg.pinv(Xb) @ y

        self.intercept_ = w[0]
        self.coef_ = w[1:]
        return self

    def predict(self, X):
        X = np.asarray(X, dtype=float)
        return self.intercept_ + X @ self.coef_


class MyRidgeRegression:
    def __init__(self, alpha=1.0):
        self.alpha = float(alpha)
        self.coef_ = None
        self.intercept_ = 0.0

    def fit(self, X, y):
        X = np.asarray(X, dtype=float)
        y = np.asarray(y, dtype=float)

        Xb = np.c_[np.ones((X.shape[0], 1)), X]
        n_features = Xb.shape[1]

        # Ridge: (X^T X + alpha*I)^-1 X^T y
        I = np.eye(n_features)
        I[0, 0] = 0.0  # свободный член не регуляризуем
        w = np.linalg.solve(Xb.T @ Xb + self.alpha * I, Xb.T @ y)

        self.intercept_ = w[0]
        self.coef_ = w[1:]
        return self

    def predict(self, X):
        X = np.asarray(X, dtype=float)
        return self.intercept_ + X @ self.coef_

print("MyLinearRegression and MyRidgeRegression defined.")


MyLinearRegression and MyRidgeRegression defined.


## 6. Обучение/оценка самописной регрессии + сравнение с sklearn

In [28]:
from sklearn.metrics import mean_absolute_error, r2_score

# --- baseline: OLS ---
my_lin = MyLinearRegression().fit(Xr_train_arr, yr_train_np)
yr_pred_my = my_lin.predict(Xr_test_arr)

my_reg_baseline_res = {
    "rmse": root_mean_squared_error(yr_test_np, yr_pred_my),
    "mae": mean_absolute_error(yr_test_np, yr_pred_my),
    "r2": r2_score(yr_test_np, yr_pred_my)
}
print("My OLS (regression) baseline:", my_reg_baseline_res)

# --- improved: Ridge (берём alpha из лучшей модели, если она есть) ---
alpha_best = 1.0
if "best_reg" in globals():
    # если best_reg - это sklearn Ridge/Lasso, пытаемся достать alpha
    try:
        alpha_best = float(best_reg.named_steps["model"].alpha)
    except Exception:
        alpha_best = 1.0

my_ridge = MyRidgeRegression(alpha=alpha_best).fit(Xr_train_arr, yr_train_np)
yr_pred_my2 = my_ridge.predict(Xr_test_arr)

my_reg_improved_res = {
    "rmse": root_mean_squared_error(yr_test_np, yr_pred_my2),
    "mae": mean_absolute_error(yr_test_np, yr_pred_my2),
    "r2": r2_score(yr_test_np, yr_pred_my2)
}
print("My Ridge (regression) improved:", my_reg_improved_res)
print("alpha used:", alpha_best)


My OLS (regression) baseline: {'rmse': 21.48576103236179, 'mae': 17.055369122574042, 'r2': 0.04239734907577464}
My Ridge (regression) improved: {'rmse': 21.48576096510958, 'mae': 17.055369062279855, 'r2': 0.04239735507052533}
alpha used: 0.01


## 7. Имплементация логистической регрессии (SGD, работает со sparse)

In [29]:
class MyLogisticRegressionSGD:
    def __init__(self, lr=0.1, epochs=15, batch_size=2048, l2=0.0, random_state=42):
        self.lr = float(lr)
        self.epochs = int(epochs)
        self.batch_size = int(batch_size)
        self.l2 = float(l2)
        self.random_state = int(random_state)

        self.w_ = None
        self.b_ = 0.0

    def fit(self, X, y):
        y = np.asarray(y).astype(float)

        n_samples, n_features = X.shape
        rng = np.random.default_rng(self.random_state)

        self.w_ = np.zeros(n_features, dtype=float)
        self.b_ = 0.0

        idx_all = np.arange(n_samples)

        for ep in range(self.epochs):
            rng.shuffle(idx_all)

            for start in range(0, n_samples, self.batch_size):
                batch_idx = idx_all[start:start+self.batch_size]
                Xb = X[batch_idx]
                yb = y[batch_idx]

                # z = Xw + b
                z = Xb.dot(self.w_) + self.b_
                p = sigmoid(z)

                # градиенты
                err = (p - yb)  # (m,)
                grad_w = (Xb.T.dot(err)) / len(batch_idx)
                grad_b = float(np.mean(err))

                # L2 регуляризация (как в ridge)
                if self.l2 > 0:
                    grad_w += self.l2 * self.w_

                # шаг
                self.w_ -= self.lr * grad_w
                self.b_ -= self.lr * grad_b

        return self

    def predict_proba(self, X):
        z = X.dot(self.w_) + self.b_
        p1 = sigmoid(z)
        p0 = 1.0 - p1
        return np.vstack([p0, p1]).T

    def predict(self, X, threshold=0.5):
        return (self.predict_proba(X)[:, 1] >= threshold).astype(int)

print("MyLogisticRegressionSGD defined.")


MyLogisticRegressionSGD defined.


## 8. Обучение/оценка самописной классификации + улучшение с регуляризацией

In [30]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# --- baseline: без регуляризации ---
my_log_base = MyLogisticRegressionSGD(lr=0.1, epochs=15, batch_size=2048, l2=0.0, random_state=RANDOM_STATE)
my_log_base.fit(Xc_train_tr, yc_train_np)

yc_pred_my = my_log_base.predict(Xc_test_tr)
yc_proba_my = my_log_base.predict_proba(Xc_test_tr)[:, 1]

my_cls_baseline_res = {
    "accuracy": accuracy_score(yc_test_np, yc_pred_my),
    "precision": precision_score(yc_test_np, yc_pred_my, zero_division=0),
    "recall": recall_score(yc_test_np, yc_pred_my, zero_division=0),
    "f1": f1_score(yc_test_np, yc_pred_my, zero_division=0),
    "roc_auc": roc_auc_score(yc_test_np, yc_proba_my),
}
print("My Logistic (classification) baseline:", my_cls_baseline_res)

# --- improved: добавляем L2 как в улучшенном бейзлайне (через лучший C, если он есть) ---
# связь простая: чем больше C, тем меньше регуляризация => l2 ~ 1/C
C_best = 1.0
if "gs_cls" in globals():
    try:
        C_best = float(gs_cls.best_params_["model__C"])
    except Exception:
        C_best = 1.0

l2_used = 1.0 / C_best

my_log_imp = MyLogisticRegressionSGD(lr=0.05, epochs=25, batch_size=2048, l2=l2_used, random_state=RANDOM_STATE)
my_log_imp.fit(Xc_train_tr, yc_train_np)

yc_pred_my2 = my_log_imp.predict(Xc_test_tr)
yc_proba_my2 = my_log_imp.predict_proba(Xc_test_tr)[:, 1]

my_cls_improved_res = {
    "accuracy": accuracy_score(yc_test_np, yc_pred_my2),
    "precision": precision_score(yc_test_np, yc_pred_my2, zero_division=0),
    "recall": recall_score(yc_test_np, yc_pred_my2, zero_division=0),
    "f1": f1_score(yc_test_np, yc_pred_my2, zero_division=0),
    "roc_auc": roc_auc_score(yc_test_np, yc_proba_my2),
}
print("My Logistic (classification) improved:", my_cls_improved_res)
print("C_best:", C_best, "| l2 used:", l2_used)


My Logistic (classification) baseline: {'accuracy': 0.8545, 'precision': 0.8552631578947368, 'recall': 0.885558583106267, 'f1': 0.8701472556894244, 'roc_auc': np.float64(0.9347557231316661)}
My Logistic (classification) improved: {'accuracy': 0.8424, 'precision': 0.8348389296062724, 'recall': 0.8897366030881018, 'f1': 0.8614139992965177, 'roc_auc': np.float64(0.9259236269181926)}
C_best: 10.0 | l2 used: 0.1


## 9. Сравнение и выводы

In [31]:
cls_compare = pd.DataFrame(
    [
        baseline_cls_res,
        improved_cls_res,
        my_cls_baseline_res,
        my_cls_improved_res
    ],
    index=["sk_baseline", "sk_improved", "my_baseline", "my_improved"]
)

reg_compare = pd.DataFrame(
    [
        baseline_reg_res,
        improved_reg_res,
        my_reg_baseline_res,
        my_reg_improved_res
    ],
    index=["sk_baseline", "sk_improved", "my_baseline", "my_improved"]
)

display(cls_compare)
display(reg_compare)


,accuracy,precision,recall,f1,roc_auc
sk_baseline,0.8652,0.870962,0.886467,0.878646,0.944571
sk_improved,0.8653,0.871117,0.886467,0.878725,0.944541
my_baseline,0.8545,0.855263,0.885559,0.870147,0.934756
my_improved,0.8424,0.834839,0.889737,0.861414,0.925924


,rmse,mae,r2
sk_baseline,21.485761,17.055369,0.042397
sk_improved,21.485204,17.054866,0.042447
my_baseline,21.485761,17.055369,0.042397
my_improved,21.485761,17.055369,0.042397


## Выводы

В задаче классификации библиотечные модели логистической регрессии из `sklearn` показали наилучшие результаты. Подбор гиперпараметров дал лишь небольшое улучшение качества, что говорит о стабильности бейзлайна. Самописная реализация работает корректно, однако по всем основным метрикам уступает библиотечной версии, а её улучшение не привело к росту качества.

В задаче регрессии различия между бейзлайном и улучшенными моделями практически отсутствуют. Значения RMSE, MAE и R² совпадают для библиотечных и самописных реализаций, что подтверждает корректность моей имплементации линейной регрессии, но также указывает на ограниченные возможности линейных моделей для данного набора данных.

В целом библиотечные реализации обеспечивают более стабильное качество и удобны для практического применения, тогда как самописные модели целесообразно использовать в учебных целях для понимания работы алгоритмов.
